In [4]:
import requests
from bs4 import BeautifulSoup


In [2]:
from io import StringIO

In [3]:
import pandas as pd
import time

In [5]:
import sys,os
sys.path.append(os.getcwd())


In [63]:
police_url = "https://haryanapolice.gov.in/ViewFIR/FIRStatusSearch?From=LFhlihlx/W49VSlBvdGc4w=="

In [64]:
pool = requests.get(police_url)

In [1]:
test_url = "https://haryanapolice.gov.in/ViewFIR/FIRStatusSearch?From=LFhlihlx%2fW49VSlBvdGc4w%3d%3d"

In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

In [99]:
service = Service("./chromedriver")
options = webdriver.ChromeOptions()
options.add_argument("--log-level=3")
options.add_argument("--headless")
driver = webdriver.Chrome(service=service, options=options)

In [67]:
driver.get(test_url)

Select(
    driver.find_element(By.NAME, "ctl00$ContentPlaceHolder1$ddFIRYear")
).select_by_value("2019")

Select(
    driver.find_element(By.NAME, "ctl00$ContentPlaceHolder1$ddlDistrict")
).select_by_value("13225")

Select(
    driver.find_element(By.NAME, "ctl00$ContentPlaceHolder1$ddlPoliceStation")
).select_by_value("13225008")

driver.find_element(
    By.XPATH,
    "//input[@name='ctl00$ContentPlaceHolder1$btnStatusSearch' and @value='Search']",
).click()

tag_present = EC.presence_of_element_located((By.ID, "ContentPlaceHolder1_RepeaterPaging"))
wait = WebDriverWait(driver, 10)
wait.until(tag_present)

fir_texts = []
first_page_data = []

page_links = driver.find_elements(
    By.XPATH, '//table[@id="ContentPlaceHolder1_RepeaterPaging"]/tbody/tr/td[@class="gridPager"]'
)
m = len(page_links)



In [97]:
Select(
    driver.find_element(By.NAME, "ctl00$ContentPlaceHolder1$ddFIRYear")
).select_by_value("2019")

Select(
    driver.find_element(By.NAME, "ctl00$ContentPlaceHolder1$ddlDistrict")
).select_by_value("13225")

all_stations = [
    x.get_attribute("value") for x in driver.find_element
    (By.NAME,"ctl00$ContentPlaceHolder1$ddlPoliceStation").find_elements(By.TAG_NAME,"option") if x != '']
print(options)

['', '13225097', '13225003', '13225007', '13225008', '13225012', '13225095', '13225098', '13225021', '13225026', '13225031', '13225035', '13225091', '13225100', '13225061', '13225096', '13225099', '13225103', '13225102', '13225101', '13225065', '13225087', '13225088', '13225089', '13225094', '13225085', '13225066', '13225090', '13225076', '13225077', '13225092', '13225093', '13225002']


In [ ]:
for ind in range(m):

    print(ind)

    if ind != 0:
        #paginate
        driver.find_element(
            By.ID, f"ContentPlaceHolder1_RepeaterPaging_Pagingbtn_{ind}"
        ).click()
        time.sleep(1)


    tag_present = EC.presence_of_element_located((By.ID, "tblDisplayRecords"))
    wait = WebDriverWait(driver, 10)
    wait.until(tag_present)

    table_links = driver.find_elements(
        By.XPATH, '//table[@id="tblDisplayRecords"]/tbody/tr'
    )

    n = len(table_links)
        

    for index in range(n):
        print(index)
        
        fir = table_links[index]

        fir.find_elements(
            By.ID, f"ContentPlaceHolder1_rptDisplayRecords_lnkFIRViewDetail_{index}"
        )[0].click()

        driver.switch_to.window(driver.window_handles[1])

        tag_present = EC.presence_of_element_located((By.NAME, "RptView$ctl06$ctl00$Next$ctl00$ctl00"))
        wait = WebDriverWait(driver, 10)
        wait.until(tag_present)

        time.sleep(1)

        first_page_content = driver.find_element(
            By.XPATH, "//td[contains(@id, 'oReportCell')]"
        ).text
        first_page_data.append(first_page_content)
        
        time.sleep(2)
        driver.find_element(
            By.XPATH, "//input[@name='RptView$ctl06$ctl00$Next$ctl00$ctl00']"
        ).click()

        tag_present = EC.presence_of_element_located((By.XPATH, "//span[contains(@class, '556')]"))
        wait = WebDriverWait(driver, 10)
        wait.until(tag_present)
        time.sleep(3)
        
        full_fir_text = driver.find_element(
            By.XPATH, "//span[contains(@class, '556')]"
        ).text
        print(full_fir_text)

        fir_texts.append(full_fir_text)

        driver.switch_to.window(driver.window_handles[0])
        # # refresh table
        table_links = driver.find_elements(
            By.XPATH, '//table[@id="tblDisplayRecords"]/tbody/tr'
        )


In [98]:
driver.close()

In [107]:
def fetch_fir_reports(test_url, driver):

    year = "2019"
    district = "13225"
    
    
    driver.get(test_url)

    Select(
        driver.find_element(By.NAME, "ctl00$ContentPlaceHolder1$ddFIRYear")
    ).select_by_value(year)

    Select(
        driver.find_element(By.NAME, "ctl00$ContentPlaceHolder1$ddlDistrict")
    ).select_by_value(district)

    all_stations = [
        str(x.get_attribute("value")) for x in driver.find_element(
            By.NAME,"ctl00$ContentPlaceHolder1$ddlPoliceStation").find_elements(By.TAG_NAME,"option") if x.get_attribute("value") != '']
    print(all_stations)
    for station in all_stations:
        print(station)
        
        fir_texts = []
        first_page_data = []

        Select(
            driver.find_element(By.NAME, "ctl00$ContentPlaceHolder1$ddlPoliceStation")
        ).select_by_value(station)
    
        driver.find_element(
            By.XPATH,
            "//input[@name='ctl00$ContentPlaceHolder1$btnStatusSearch' and @value='Search']",
        ).click()
    
        tag_present = EC.presence_of_element_located((By.ID, "ContentPlaceHolder1_RepeaterPaging"))
        wait = WebDriverWait(driver, 10)
        wait.until(tag_present)
    
        page_links = driver.find_elements(
            By.XPATH, '//table[@id="ContentPlaceHolder1_RepeaterPaging"]/tbody/tr/td[@class="gridPager"]'
        )
        m = len(page_links)
    
        for ind in range(m):
    
            print(ind)
    
            if ind != 0:
                #paginate
                driver.find_element(
                    By.ID, f"ContentPlaceHolder1_RepeaterPaging_Pagingbtn_{ind}"
                ).click()
                time.sleep(1)
    
            tag_present = EC.presence_of_element_located((By.ID, "tblDisplayRecords"))
            wait = WebDriverWait(driver, 10)
            wait.until(tag_present)
    
            table_links = driver.find_elements(
                By.XPATH, '//table[@id="tblDisplayRecords"]/tbody/tr'
            )
        
            n = len(table_links)
    
            for index in range(n):
                print(index)
                
                fir = table_links[index]
        
                fir.find_elements(
                    By.ID, f"ContentPlaceHolder1_rptDisplayRecords_lnkFIRViewDetail_{index}"
                )[0].click()
        
                driver.switch_to.window(driver.window_handles[1])
        
                tag_present = EC.presence_of_element_located((By.NAME, "RptView$ctl06$ctl00$Next$ctl00$ctl00"))
                wait = WebDriverWait(driver, 10)
                wait.until(tag_present)
        
                time.sleep(1)
        
                first_page_content = driver.find_element(
                    By.XPATH, "//td[contains(@id, 'oReportCell')]"
                ).text
                first_page_data.append(first_page_content)
                
                time.sleep(1)
                driver.find_element(
                    By.XPATH, "//input[@name='RptView$ctl06$ctl00$Next$ctl00$ctl00']"
                ).click()
        
                tag_present = EC.presence_of_element_located((By.XPATH, "//span[contains(@class, '556')]"))
                wait = WebDriverWait(driver, 10)
                wait.until(tag_present)
                time.sleep(5)
                
                full_fir_text = driver.find_element(
                    By.XPATH, "//span[contains(@class, '556')]"
                ).text
                # print(full_fir_text)
        
                fir_texts.append(full_fir_text)
        
                driver.switch_to.window(driver.window_handles[0])
                # refresh table
                table_links = driver.find_elements(
                    By.XPATH, '//table[@id="tblDisplayRecords"]/tbody/tr'
                )

        loc_corpus = pd.DataFrame(zip(fir_texts, first_page_data), columns=["fir_text", "metadata_html"])
        loc_corpus["district"] = district
        loc_corpus["station"] = station
        loc_corpus.to_csv(f"{district}_{station}_fir.csv")
    
    return 0


In [108]:
fetch_fir_reports(test_url, driver)

['13225097', '13225003', '13225007', '13225008', '13225012', '13225095', '13225098', '13225021', '13225026', '13225031', '13225035', '13225091', '13225100', '13225061', '13225096', '13225099', '13225103', '13225102', '13225101', '13225065', '13225087', '13225088', '13225089', '13225094', '13225085', '13225066', '13225090', '13225076', '13225077', '13225092', '13225093', '13225002']
13225097
0
0
1
2
3
4
5
6
7
8
9
1
0
1
2
3
4
5
6
7
8
9
2
0
1
2
3
4
5
6
7
8
9
3
0
1
2
3
4
5
6
7
8
9
4
0
1
2
3
4
5
6
7
8
9
5
0
1
2
3
4
5
6
7
8
9
6
0
1
2
3
4
5
6
7
8
9
7
0
1
2
3
4
5
6
7
8
9
8
0
1
2
3
4
5
6
7
8
9
9
0
1
2
3
4
5
6
7
8
9
10
0
1
2
3
4
5
6
7
8
9
11
0
1
2
3
4
5
6
7
8
9
12
0
1
2
3
4
5
6
7
8
9
13
0
1
2
3
4
5
6
7
8
9
14
0
1
2
3
4
5
6
7
8
9
15
0
1
2
3
4
5
6
7
8
9
16
0
1
2
3
4
5
6
7
8
9
17
0
1
2
3
4
5
6
7
8
9
18
0
1
2
3
4
5
6
7
8
9
19
0
1
2
3
4
5
6
7
8
9
20
0
1
2
3
4
5
6
7
8
9
21
0
1
2
3
4
5
6
7
8
9
22
0
1
2
3
4
5
6
7
8
9
23
0
1
2
3
4
5
6
7
8
9
24
0
1
2
3
4
5
6
7
8
9
25
0
1
2
3
4
5
6
7
8
9
26
0
1
2
3
4
5
6
7

StaleElementReferenceException: Message: stale element reference: stale element not found
  (Session info: chrome-headless-shell=123.0.6312.59); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
0   chromedriver                        0x0000000104f6d2a8 chromedriver + 4625064
1   chromedriver                        0x0000000104f64e53 chromedriver + 4591187
2   chromedriver                        0x0000000104b63e6a chromedriver + 392810
3   chromedriver                        0x0000000104b69394 chromedriver + 414612
4   chromedriver                        0x0000000104b6b12b chromedriver + 422187
5   chromedriver                        0x0000000104b6b1cc chromedriver + 422348
6   chromedriver                        0x0000000104baf227 chromedriver + 700967
7   chromedriver                        0x0000000104baf5b1 chromedriver + 701873
8   chromedriver                        

In [ ]:
driver.close()

In [385]:
[x for x in fir_texts if "25-" in x]

['सेवा मे, श्रीमान SHO साहब थाना भुपानी ‘जय हिंद’ आज मन HC मय सि0 सोनु 2785 व मय गाडी सरकारी HR-51 BM – 8073 टाटा सुमो जिसका चालक CT. जगदीप 1205 है के बराये गश्त व पडताल क्राईम खेडी पुल-नचौली रोड भुपानी मोड पर मौजुद हुँ कि एक नौजवान लडका गांव भुपानी की तरफ से पैदल आता हुआ दिखाई दिया जो सामने खडी पुलिस पार्टी को देखकर एक दम वापिस मुडकर तेज कदमो से चलने लगा। मन HC ने भागकर साथी CT की मदद से शक की बिनाह पर शख्स मजकुरा को काबु करके नाम पता पुछा तो जिसने अपना नाम रोहित S/O स्व0 जितेन्द्र भारती R/O रामनगर कालोनी भुपानी मोड थाना भुपानी जिला फरीदाबाद बा उम्र 22 वर्ष बतलाया। तलाशी लेने पर जिसकी पहनी हुई पैंट जींस की दाहिनी डब से एक देशी पिस्तौल 32 बोर बरामद हुआ। चैक करने पर जो खाली मिला। शख्स मजकुरा अपने पास देशी पिस्तौल रखने बारे मांगने पर कोई लाईंसेंस पेश नही कर सका। देशी पिस्तौल का अलग के खाका तैयार करके लाईसेंस पेश नही कर सका। देशी पिस्तौल का अलग से खाका तैयार करके पैमाइश करने पर स्लाईड बैरल ल0 14 C.M, बट ल0 8 CM व कुल लम्बाई 18 C.M हुई। बरामदा शुदा देशी पिस्तौल का अलग से पलंदा तैयार करके प

In [394]:
fir_texts[2]

'सेवा मे, श्रीमान SHO साहब थाना भुपानी ‘जय हिंद’ आज मन HC मय सि0 सोनु 2785 व मय गाडी सरकारी HR-51 BM – 8073 टाटा सुमो जिसका चालक CT. जगदीप 1205 है के बराये गश्त व पडताल क्राईम खेडी पुल-नचौली रोड भुपानी मोड पर मौजुद हुँ कि एक नौजवान लडका गांव भुपानी की तरफ से पैदल आता हुआ दिखाई दिया जो सामने खडी पुलिस पार्टी को देखकर एक दम वापिस मुडकर तेज कदमो से चलने लगा। मन HC ने भागकर साथी CT की मदद से शक की बिनाह पर शख्स मजकुरा को काबु करके नाम पता पुछा तो जिसने अपना नाम रोहित S/O स्व0 जितेन्द्र भारती R/O रामनगर कालोनी भुपानी मोड थाना भुपानी जिला फरीदाबाद बा उम्र 22 वर्ष बतलाया। तलाशी लेने पर जिसकी पहनी हुई पैंट जींस की दाहिनी डब से एक देशी पिस्तौल 32 बोर बरामद हुआ। चैक करने पर जो खाली मिला। शख्स मजकुरा अपने पास देशी पिस्तौल रखने बारे मांगने पर कोई लाईंसेंस पेश नही कर सका। देशी पिस्तौल का अलग के खाका तैयार करके लाईसेंस पेश नही कर सका। देशी पिस्तौल का अलग से खाका तैयार करके पैमाइश करने पर स्लाईड बैरल ल0 14 C.M, बट ल0 8 CM व कुल लम्बाई 18 C.M हुई। बरामदा शुदा देशी पिस्तौल का अलग से पलंदा तैयार करके पि

In [395]:
first_page_data[2]

"HARYANA POLICE CITIZEN SERVICES (हरियाणा पुलिस नागरिक सेवा)\n    FIRST INFORMATION REPORT\n         (Under Section 154 Cr.P.C.)\n        प्रथम सूचना रिपोर्ट\n (धारा 154 दंड प्रक्रिया सहिंता के तहत)\n1.\nDistrict (ज़िला): FARIDABAD\nP.S. (थाना): BHUPANI\nYear (वर्ष): 2019\nFIR No. (प्र.सू.रि. सं.): 0003\nDate (दिनांक): 05/01/2019 22:51\n2.\nS.No. (क्र.सं.)\nActs (अधिनियम)\nSections (धारा(एँ))\n1\nARMS ACT, 1959\n25\n2\nARMS ACT, 1959\n54\n3\nARMS ACT, 1959\n59\n3.\n(a)\nOccurrence of offence (अपराध की घटना):\nDate from (दिनांक से): \n1\nDay (दिन): \nDate To (दिनांक तक): \nTime Period (समय अवधि): \nTime To (समय तक): \nTime From (समय से): \n(b)\nInformation received at P.S. (थाना जहां सूचना प्राप्त हुई):\nDate (दिनांक): 05/01/2019\nTime (समय): 18:30 hrs\n(c)\nGeneral Diary Reference (रोजनामचा संदर्भ): \nEntry No. (प्रविष्टि सं.):  020\nTime (समय): 05/01/2019 22:44 hrs\n4.\nType of Information (सूचना का प्रकार): Written\n5.\nPlace of Occurrence (घटनास्थल): \n1.\n(a) Direction and distance 

In [381]:
first_page_data

["HARYANA POLICE CITIZEN SERVICES (हरियाणा पुलिस नागरिक सेवा)\n    FIRST INFORMATION REPORT\n         (Under Section 154 Cr.P.C.)\n        प्रथम सूचना रिपोर्ट\n (धारा 154 दंड प्रक्रिया सहिंता के तहत)\n1.\nDistrict (ज़िला): FARIDABAD\nP.S. (थाना): BHUPANI\nYear (वर्ष): 2019\nFIR No. (प्र.सू.रि. सं.): 0001\nDate (दिनांक): 03/01/2019 20:17\n2.\nS.No. (क्र.सं.)\nActs (अधिनियम)\nSections (धारा(एँ))\n1\nTHE PUBLIC GAMBLING ACT, 1867, 3 OF 1867\n13\n3.\n(a)\nOccurrence of offence (अपराध की घटना):\nDate from (दिनांक से): 03/01/2019\n1\nDay (दिन): Thursday\nDate To (दिनांक तक): 03/01/2019\nTime Period (समय अवधि): Pahar 6\nTime To (समय तक): 16:50 hrs\nTime From (समय से): 16:05 hrs\n(b)\nInformation received at P.S. (थाना जहां सूचना प्राप्त हुई):\nDate (दिनांक): 03/01/2019\nTime (समय): 17:30 hrs\n(c)\nGeneral Diary Reference (रोजनामचा संदर्भ): \nEntry No. (प्रविष्टि सं.):  018\nTime (समय): 03/01/2019 19:54 hrs\n4.\nType of Information (सूचना का प्रकार): Written\n5.\nPlace of Occurrence (घटनास्थल):

In [69]:
corpus = pd.DataFrame(zip(fir_texts, first_page_data), columns=["fir_text", "metadata_html"])
corpus.to_csv("test_files.csv")

In [81]:
corpus[corpus["fir_text"] == "FARIDABAD"]["metadata_html"][11] 

"HARYANA POLICE CITIZEN SERVICES (हरियाणा पुलिस नागरिक सेवा)\n    FIRST INFORMATION REPORT\n         (Under Section 154 Cr.P.C.)\n        प्रथम सूचना रिपोर्ट\n (धारा 154 दंड प्रक्रिया सहिंता के तहत)\n1.\nDistrict (ज़िला): FARIDABAD\nP.S. (थाना): BHUPANI\nYear (वर्ष): 2019\nFIR No. (प्र.सू.रि. सं.): 0012\nDate (दिनांक): 17/01/2019 12:05\n2.\nS.No. (क्र.सं.)\nActs (अधिनियम)\nSections (धारा(एँ))\n1\nIPC 1860\n174-A\n3.\n(a)\nOccurrence of offence (अपराध की घटना):\nDate from (दिनांक से): \n1\nDay (दिन): \nDate To (दिनांक तक): \nTime Period (समय अवधि): \nTime To (समय तक): \nTime From (समय से): \n(b)\nInformation received at P.S. (थाना जहां सूचना प्राप्त हुई):\nDate (दिनांक): 17/01/2019\nTime (समय): 10:30 hrs\n(c)\nGeneral Diary Reference (रोजनामचा संदर्भ): \nEntry No. (प्रविष्टि सं.):  011\nTime (समय): 17/01/2019 11:41 hrs\n4.\nType of Information (सूचना का प्रकार): Written\n5.\nPlace of Occurrence (घटनास्थल): \n1.\n(a) Direction and distance from P.S. (थाना से दूरी और दिशा):  \nBeat No. (बी

In [89]:
corpus[corpus["fir_text"] == "FARIDABAD"]["metadata_html"][31]

"HARYANA POLICE CITIZEN SERVICES (हरियाणा पुलिस नागरिक सेवा)\n    FIRST INFORMATION REPORT\n         (Under Section 154 Cr.P.C.)\n        प्रथम सूचना रिपोर्ट\n (धारा 154 दंड प्रक्रिया सहिंता के तहत)\n1.\nDistrict (ज़िला): FARIDABAD\nP.S. (थाना): BHUPANI\nYear (वर्ष): 2019\nFIR No. (प्र.सू.रि. सं.): 0032\nDate (दिनांक): 13/03/2019 08:30\n2.\nS.No. (क्र.सं.)\nActs (अधिनियम)\nSections (धारा(एँ))\n1\nARMS ACT, 1959\n25\n2\nARMS ACT, 1959\n54\n3\nARMS ACT, 1959\n59\n3.\n(a)\nOccurrence of offence (अपराध की घटना):\nDate from (दिनांक से): 12/03/2019\n1\nDay (दिन): Tuesday\nDate To (दिनांक तक): 12/03/2019\nTime Period (समय अवधि): Pahar 7\nTime To (समय तक): 21:00 hrs\nTime From (समय से): 18:00 hrs\n(b)\nInformation received at P.S. (थाना जहां सूचना प्राप्त हुई):\nDate (दिनांक): 13/03/2019\nTime (समय): 00:15 hrs\n(c)\nGeneral Diary Reference (रोजनामचा संदर्भ): \nEntry No. (प्रविष्टि सं.):  004\nTime (समय): 13/03/2019 07:57 hrs\n4.\nType of Information (सूचना का प्रकार): Written\n5.\nPlace of Occu

In [377]:
driver.close()